# [E-08] Movielens 영화 추천 실습

## 목차
1. 데이터 준비와 전처리
2. 데이터 탐색
3. CSR Matrix
4. MF 모델 학습
5. 비슷한 영화 찾기
6. 내가 좋아할만한 영화 추천

# 루브릭 평가기준
1. CSR matrix가 정상적으로 만들어졌다. 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.


* 첫번째: 6039*3631 - 메트릭스 크기가 안맞아서 오류
* 두번째: 6041*3958 -


2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.


* 수치 계산은 됐으나 높진않다. factors, iterations 를 몇번 변경해도 0.01을 넘지 않았다 (0.014919596)
* 원인은 선호하는 영화 5가지가 개연성이 없어서 그런것으로 판단하고 액션 영화 위주로 변경했지만 딱히 유사성이 높은 영화는 아닌가보다...(0.2888981)


3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.


 비슷한 영화와 내가 좋아할 만한 영화는 꽤 근접하게 추천하는거 같다. 

* terminator와 비슷한 영화:  'killer','woman in the dunes'
* men in black 선호 유저가 좋아할만한 영화: 
star wars: episode vi - return of the jedi','alien'


# 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/aiffel_exp_data/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'view_count'}, inplace=True)
ratings['view_count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: view_count, Length: 836478, dtype: int64

In [4]:
#timestemp는 필요없는 칼럼이라 삭제 
ratings_cols = ['user_id', 'movie_id','view_count']
ratings = ratings[ratings_cols]
ratings.head(10)

,user_id,movie_id,view_count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [5]:
ratings.head(5)

,user_id,movie_id,view_count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/aiffel_exp_data/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# # 사용하는 컬럼만 남겨줍니다. 장르는 필요가 없는거 같음
cols = ['movie_id', 'title'] 
movies = movies[cols]
movies.head(10)

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [8]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title
0,1,toy story (1995)
1,2,jumanji (1995)
2,3,grumpier old men (1995)
3,4,waiting to exhale (1995)
4,5,father of the bride part ii (1995)
5,6,heat (1995)
6,7,sabrina (1995)
7,8,tom and huck (1995)
8,9,sudden death (1995)
9,10,goldeneye (1995)


In [9]:
# 두 데이터프레임 병합! 한번에 보자
rating= pd.merge(ratings,movies)
rating

,user_id,movie_id,view_count,title
0,1,914,3,my fair lady (1964)
1,6,914,5,my fair lady (1964)
2,10,914,5,my fair lady (1964)
3,33,914,5,my fair lady (1964)
4,35,914,3,my fair lady (1964)
...,...,...,...,...
834350,5851,3607,5,one little indian (1973)
834351,5854,3026,4,slaughterhouse (1987)
834352,5854,690,3,"promise, the (versprechen, das) (1994)"
834353,5938,2909,4,"five wives, three secretaries and me (1998)"


In [10]:
rating.loc[834355]=[6041,2910,5,'star wars: episode iv - a new hope (1977)']  
rating.loc[834356]=[6041,2885,4,'star wars: episode v - the empire strikes back (1980)']
rating.loc[834357]=[6041,2434,5,'matrix, the (1999)']
rating.loc[834358]=[6041,2019,4,'terminator, the (1984)']
rating.loc[834359]=[6041,2297,5,'men in black (1997)']
rating

,user_id,movie_id,view_count,title
0,1,914,3,my fair lady (1964)
1,6,914,5,my fair lady (1964)
2,10,914,5,my fair lady (1964)
3,33,914,5,my fair lady (1964)
4,35,914,3,my fair lady (1964)
...,...,...,...,...
834355,6041,2910,5,star wars: episode iv - a new hope (1977)
834356,6041,2885,4,star wars: episode v - the empire strikes back...
834357,6041,2434,5,"matrix, the (1999)"
834358,6041,2019,4,"terminator, the (1984)"


# 2) 데이터 탐색

In [11]:
#유저수
rating['user_id'].nunique()

6040

In [12]:
#영화 수 
rating['movie_id'].nunique()

3627

In [13]:
#인기있는 영화 30개 
popular_movie = rating.groupby('title')['movie_id'].count()
popular_movie.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2911
star wars: episode v - the empire strikes back (1980)    2886
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2435
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2298
schindler's list (1993)                                  2257
pr

In [14]:
# 유저별 몇개의 영화를 보고 있는지에 대한 통계
user_count = rating.groupby('user_id')['title'].count()
user_count.describe()

count    6040.000000
mean      138.139073
std       156.009227
min         1.000000
25%        37.000000
50%        81.000000
75%       176.000000
max      1966.000000
Name: title, dtype: float64

In [15]:
# 유저별 평점의 평균
user_mean = rating.groupby('user_id')['view_count'].mean()
user_mean 

user_id
1       4.196078
2       3.913043
3       4.130435
4       4.473684
5       3.720280
          ...   
6037    3.845745
6038    4.055556
6039    3.949153
6040    4.047273
6041    4.600000
Name: view_count, Length: 6040, dtype: float64

In [16]:
#결측치
rating.isnull().sum()

user_id       0
movie_id      0
view_count    0
title         0
dtype: int64

In [17]:
#중복데이터
rating.duplicated().sum()

0

In [18]:
#결측치와 중복데이터 모두 없음
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 834360 entries, 0 to 834359
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     834360 non-null  int64 
 1   movie_id    834360 non-null  int64 
 2   view_count  834360 non-null  int64 
 3   title       834360 non-null  object
dtypes: int64(3), object(1)
memory usage: 31.8+ MB


* 어려웠던 부분!

모델 학습 목적: 영화에 대한 나의 선호도를 예측하고 선호하는 영화나 비슷한 영화를 추천하는 것!!   이때 영화이름을 idx로 바꿔서 함수를 적용하고 다시 이름으로 출력한다. 

Problem 1. 무엇을 인덱싱할건가?

-user id : user id는 숫자로 표시되서 인덱싱 하지 않아도 된다. 

-movie id: 결국 title을 쳐서 예측하기 때문에 csr matrix 에서만 사용한다. 

-title : str인 제목을 숫자로 바꿔서 계산 후 다시 제목으로 출력해야해서 inx 필요 

Problem 2. inx to someting도 하고 someting to inx도 하고 이 과정이 복잡해보인다. 
이 과정에서 흐름이 계속 헷갈려서 헤맸다.

Solution: @shate 도움을 받아 movie id 넣으면 제목 나오도록 딕셔너리를 만들었다. 인덱싱을 만들지 않아도 제목을 치면 딕셔너리에서 찾을 수 있다!

In [19]:
idx_to_title = {} #movie id -> 제목
title_to_idx = {} #제목 -> movie id   

for i in range(rating.shape[0]): 
    idx_to_title[rating['movie_id'][i]] = rating['title'][i] 
    title_to_idx[rating['title'][i]] = rating['movie_id'][i] 
    
print(idx_to_title[300]) 
print(title_to_idx['erin brockovich (2000)'])   

quiz show (1994)
3408


# 3) CSR matrix

In [20]:
rating['user_id'].max()

6041

In [21]:
from scipy.sparse import csr_matrix

num_user_id = rating['user_id'].nunique() #사용자
num_movie_id = rating['movie_id'].nunique() # 유저

csr_data = csr_matrix((rating.view_count,(rating.user_id, rating.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 834360 stored elements in Compressed Sparse Row format>

* Error

1.row index exceeds matrix dimensions : row을 user_id로 했을때 변수 행렬 범위에 벗어난 값을 출력한다고 오류 

try 1: 6039*3631 sparse matrix 

problem: self.row.max() >= self.shape[0] self.cal.max() >= self.shape[1] 라서 더 커서 shape크기가 안맞아서 오류가 났다.

solution: csr_data 뒤에 있던 shape(m,n) 를 삭제 -> 12001x3953 로 출력됐다. 이후 self.row.max() < self.shape 로 크기가 더 커져서 오류가 사라졌다 (helper @vg-rlo)

2.invalid literal for int() with base 10: 'my fair lady (1964)' columns가 title일 경우 형변환 오류 

정수는 정수로 받고 실수는 실수로 받고 문자는 문자로 받는데 지금 형이 일치하지 않음 imput으로 받은 값이 문자형이라서 문자형을 실수로 바꾸고 정수로 변경

# 4) als_model = AlternatingLeastSquares 모델훈련

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#factor와 iteration을 늘릴 수록 잘 학습하지만 과적합될수 있음
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, #유저와 아이템벡터의차원
                                    regularization=0.01,  #과적합방지를 이한 정규화값
                                    use_gpu=False,        # gpu사용
                                    iterations=10, dtype=np.float32) #epochs 반복

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
#모델이 사용자벡터와 영화의 백터를 어떻게 만들었는지 확인
sumin, movie = 6041, title_to_idx['star wars: episode v - the empire strikes back (1980)']
sumin_vector, movie_vector = als_model.user_factors[sumin], als_model.item_factors[movie]

print('슝=3')

슝=3


In [24]:
sumin_vector

array([-0.3888236 ,  0.5896644 ,  0.21428992,  0.16490017, -0.05473063,
        0.05973079,  0.07999105, -0.17540836, -0.253804  ,  0.41749066,
       -0.16767257, -0.07912429,  0.45948285,  0.11757493,  0.08285497,
        0.05381104, -0.1335893 ,  0.13839428, -0.08688626,  0.46000937,
        0.6653447 ,  0.20446235,  0.02283026, -0.34636533,  0.1461531 ,
        0.4877925 ,  0.39745837,  0.10971488, -0.43748668,  0.23859979,
       -0.54502904,  0.4023337 , -0.07059328, -0.40070513, -0.03317815,
       -0.01359612,  0.45010605,  0.5039563 ,  0.13573167, -0.0260837 ,
        0.3587274 , -0.21731298, -0.00873773,  0.33375272, -0.3158376 ,
       -0.46835974, -0.10209564, -0.2065195 , -0.33888257,  0.6004863 ,
        0.7933488 ,  0.03544756,  0.1721634 , -0.16877554, -0.36223283,
       -0.01155066, -0.00904679, -0.03767988,  0.20643425, -0.01500396,
       -0.5798486 ,  0.54948646, -0.4747599 , -0.56938493, -0.38354108,
       -0.11306174, -0.27361554, -0.3901433 ,  0.13179365, -0.19

In [25]:
movie_vector

array([ 2.2292119e-03,  6.8464540e-03,  1.5619591e-02,  4.2778137e-03,
        4.2075817e-03, -4.5854906e-03,  5.4737171e-03,  3.0868076e-04,
        1.7862442e-03, -6.4446460e-03,  7.2054900e-03,  5.6679975e-03,
        6.2411050e-03,  6.3142814e-03,  5.2519105e-03,  9.1189065e-04,
        4.4565219e-03,  9.4941193e-03,  5.4165497e-03,  1.4298118e-02,
        1.0274077e-02,  3.0884301e-04, -1.4330163e-03,  8.1185093e-03,
        2.1042773e-03,  7.5253281e-03,  8.3342735e-03,  3.8955128e-04,
        1.1156066e-02,  1.5723320e-02,  7.1930136e-03,  1.4615632e-02,
        1.5731695e-03,  2.8208706e-03,  2.1553179e-03,  2.1688000e-03,
       -1.4137760e-04,  5.3674374e-03,  1.4268139e-02,  8.9653740e-03,
        8.6475387e-03,  6.8604259e-04,  2.9340945e-04,  1.8882612e-02,
        7.9969835e-04,  1.0332503e-02,  1.1237716e-02,  1.1097363e-02,
        7.1030222e-03,  4.6296529e-03,  1.7743825e-03,  1.6068470e-02,
        5.2806330e-03,  4.9711424e-03,  4.7642984e-03,  1.5145855e-02,
      

In [26]:
#모델이 예측한 men in black (1997)에 대한 나의 선호도
np.dot(sumin_vector,movie_vector) 

0.038157262

In [27]:
# 모델이 예측한 terminator, the (1984)에 대한 나의 선호도
terminator = title_to_idx['terminator, the (1984)']
terminator_vector = als_model.item_factors[terminator]
np.dot(sumin_vector,terminator_vector)

0.33058432

# 5) 비슷한 영화 찾기

* Error

비슷한 영화를 찾을때 내가 추가한 영화를 넣으면 그런 영화는 없다고 오류가 났다.
하지만, 기존에 있는 영화제목을 넣으면 오류가 나지 않았다. 

* Problem

사용자 초기 세팅할때 좋아하는 영화를 5개 추가하는 부분이었다.
영화 이름을 데이터셋 안에 있는걸로 했어야 하는데 나는 완전 새로운 영화 제목을 추가해서 기존 데이터와 나의 선호데이터가 분리돼서 
기존 데이터에는 나의 선호 데이터가 없는게 당연했다!

* Solution

전처리를 다시!! 기존에 있는 영화제목으로 loc함수 사용해서 추가했다. 

In [28]:
def get_similar_movie(title_name: str):
    title = title_to_idx[title_name]
    similar_movie = als_model.similar_items(title)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [29]:
get_similar_movie('terminator, the (1984)')

['terminator, the (1984)',
 'yojimbo (1961)',
 'kagemusha (1980)',
 'ran (1985)',
 'seventh seal, the (sjunde inseglet, det) (1957)',
 'sanjuro (1962)',
 'dersu uzala (1974)',
 '8 1/2 (1963)',
 'solaris (solyaris) (1972)',
 'killer, the (die xue shuang xiong) (1989)']

# 6) 내가 좋아할만한 영화 추천

In [30]:
user = title_to_idx[ 'men in black (1997)']
# recommend에서는 user*item CSR Matrix를 가져온다
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(1214, 0.8082917),
 (1210, 0.796536),
 (318, 0.7236809),
 (1233, 0.7142164),
 (1203, 0.70045984),
 (1387, 0.6657398),
 (1240, 0.65527207),
 (1036, 0.6539483),
 (3654, 0.61505026),
 (589, 0.5800204),
 (2762, 0.5641844),
 (924, 0.55715734),
 (1198, 0.556236),
 (1272, 0.5448681),
 (2628, 0.5204387),
 (1252, 0.51032317),
 (608, 0.50076205),
 (1225, 0.49698567),
 (1287, 0.4867663),
 (2194, 0.48439515)]

In [31]:
[idx_to_title[i[0]] for i in artist_recommended]

['alien (1979)',
 'star wars: episode vi - return of the jedi (1983)',
 'shawshank redemption, the (1994)',
 'boat, the (das boot) (1981)',
 '12 angry men (1957)',
 'jaws (1975)',
 'terminator, the (1984)',
 'die hard (1988)',
 'guns of navarone, the (1961)',
 'terminator 2: judgment day (1991)',
 'sixth sense, the (1999)',
 '2001: a space odyssey (1968)',
 'raiders of the lost ark (1981)',
 'patton (1970)',
 'star wars: episode i - the phantom menace (1999)',
 'chinatown (1974)',
 'fargo (1996)',
 'amadeus (1984)',
 'ben-hur (1959)',
 'untouchables, the (1987)']

In [32]:
#matrix, the (1999)가 추천된 이유
matrix = title_to_idx['rocky (1976)']
explain = als_model.explain(user, csr_data, itemid=matrix)

In [33]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('french connection, the (1971)', 0.10017395350765543),
 ('hunt for red october, the (1990)', 0.07680154533525602),
 ('goldfinger (1964)', 0.0660293364113134),
 ('close encounters of the third kind (1977)', 0.06551934752366746),
 ('saving private ryan (1998)', 0.054002204597100804),
 ('glory (1989)', 0.05369663758215477),
 ('animal house (1978)', 0.05335664698206202),
 ('papillon (1973)', 0.04950863422395),
 ('godfather: part ii, the (1974)', 0.037177133851154995),
 ('fugitive, the (1993)', 0.036090641469344785)]

# 회고

* 알게된 개념 


1.추천시스템 모델 중 MF(행렬분해)모델을 사용 

행렬분해는 (m,n) 사이즈의 행렬R 을 (m,k)사이즈 행렬P과 (k,n) 사이즈 행렬Q로 분해한다.
두 행렬의 벡터를 내적해서 얻은 값이 R과 같은지(비슷한지) 계산한다. 

E8을 기반으로 보면 아래를 계산하는 것
M = 유저수(사용자 벡터) N = 영화수(영화벡터) P와 Q의 내적값(P*Q) = R(유저가 영화를 평가한 수치:별점) 

2.CSR Matrix 
* Sparse Matrix

P*Q 는 유저가 보지 않은 영화의 정보까지 행렬에 0으로 포함되어 있 다. 메모리만 늘어나고 필요는 없다! (Sparse Matrix) 

* CSR Matrix

그래서 유저가 본 영화 정보의 값만 인덱스를 표시해서 행렬로 만든다
(sparse matrix를 압축해서 유효한 데이터의 값과 데이터가 있는 위치만으로 된 행렬을 만든다.)

3.마무리

 가장 자주 접하는 기술인 추천시스템을 직접 만들어봐서 신기했다. 
실제로는 고려해야할 column이 더 많겠지만 흐름은 의외로 이해하기 쉬웠다. 

아직 복잡한 흐름이 나오면 헷갈려하지만 명절에 노드를 복습하면서 기본적인 함수는 이해해서 이전보다 더 잘 이해하면서 진행했다